In [1]:
import re
from nltk.tokenize import sent_tokenize

In [2]:
def Read_Corpus(path):
    corpus_file = open(path, 'r')
    lines = corpus_file.readlines()
    text = ""
    for line in lines:
        striped_line = line.strip()
        if striped_line != '':
            text = text + " " + striped_line
    return text


def substitue_tokens(sentences):
    sub_sentences = []
    for sentence in sentences:
        sub_sen = sentence
        sub_sen = re.sub("#[a-zA-Z0-9_]+", "<HASHTAG>", sub_sen)
        sub_sen = re.sub("@[a-zA-Z0-9_]+", "<MENTION>", sub_sen)
        sub_sen = re.sub("https?://[a-zA-Z0-9_./]+", "<URL>", sub_sen)
        sub_sentences.append(sub_sen)
    return sub_sentences


def split_corpus_into_sentences(corpus):
    sentences = sent_tokenize(corpus)
    return sentences


def tokenize_sentence(sen):
    # Handles words 
    word_wise = sen.split()

    # Handeling punctuations
    tokenized_sen = []
    for word in word_wise:
        if(is_punc(word) == True):
            tokenized_sen.append(word)
        else:
            var_word = word
            while(starting_punc(var_word) == True):
                tokenized_sen.append(word[0])
                var_word = word[1:]
            
            end_puncs = []
            while(ending_punc(var_word) == True):
                end_puncs = [var_word[-1]] + end_puncs
                var_word = var_word[:-1]
            
            tokenized_sen.append(var_word)
            tokenized_sen += end_puncs
    
    return tokenized_sen


# Does the given word have punctation at the end
def ending_punc(word):
    if(word[-1] == ',') or (word[-1] == ':') or (word[-1] == ';') or (word[-1] == '"') or (word[-1] == ')') or (word[-1] == '}') or (word[-1] == ']') or (word[-1] == '.') or (word[-1] == '?') or (word[-1] == '!'):
        return True
    else:
        return False
    

# Does the given word have punctation at the start
def starting_punc(word):
    if(word[0] == '"') or (word[0] == '(') or (word[0] == '{') or (word[0] == '['):
        return True
    else:
        return False
    

# Is the given word a punctation
def is_punc(word):
    if(len(word) == 1 and (ending_punc(word) or starting_punc(word))):
        return True
    else:
        return False

In [3]:
corpus = Read_Corpus('test.txt')
sentences = sent_tokenize(corpus)
url_metions = substitue_tokens(sentences)
sentence_tokenized = []
for sentence in url_metions:
    tokenized_sen = tokenize_sentence(sentence)
    sentence_tokenized.append(tokenized_sen)
for sentence in sentence_tokenized:
    print(sentence)

['<MENTION>', '<HASHTAG>', '<URL>', 'are', 'my', 'subtitions', 'working', '.']
['Is', 'the', 'split', 'based', 'on', 'sentece', 'working', '?']
['yes', ',', 'it', 'is', 'working', '?', '?', '?', '!']
['yeah', 'bro', '.']


In [4]:
def ngrams_from_tokens_sen(tokens, ngram_size, Counts):
    ngram = ["w" for i in range(ngram_size)]
    for i in range(len(tokens)):
        for j in range(ngram_size - 1):
            ngram[j] = ngram[j + 1]
        ngram[ngram_size - 1] = tokens[i]
        
        history = tuple((ngram[i] for i in range(ngram_size - 1)))
        word = ngram[ngram_size - 1]

        if history in Counts.keys():
            if word in Counts[history].keys():
                Counts[history][word] += 1
            else:
                Counts[history][word] = 1
        else:
            Counts[history] = {}
            Counts[history][word] = 1


def unigram_from_tokens_sen(tokens, Counts):
    for token in tokens:
        if token in Counts.keys():
            Counts[token] += 1
        else:
            Counts[token] = 1


def sum_values(Counts, ngram_size):
    SumVal = {}
    if(ngram_size == 1):
        SumVal["SUM"] = sum(Counts.values())
    else:
        for history in Counts.keys():
            SumVal[history] = sum(Counts[history].values())
    return SumVal


# extract from bigram
def unigram_keser_nay_data(Counts):
    keser_nay_unigram = {}
    
    total_unique_bigrams = 0
    for history in Counts.keys():
        total_unique_bigrams += len(Counts[history].keys())
    keser_nay_unigram["total_bigrams"] = total_unique_bigrams
    
    for history in Counts.keys():
        for word in Counts[history].keys():
            if word in keser_nay_unigram.keys():
                keser_nay_unigram[word] += 1
            else:
                keser_nay_unigram[word] = 1
    return keser_nay_unigram
    
                
def conditional_prob(history, word, Counts, SumCounts):
    if history in Counts.keys():
        if word in Counts[history].keys():
            return Counts[history][word] / SumCounts[history]
        else:
            return 0
    else:
        return 0
    

def unigram_prob(word, Counts, SumCounts):
    if word in Counts.keys():
        return Counts[word] / SumCounts["SUM"]
    else:
        return 0

In [5]:
tokens = ["Hello", "world", ",", "today", "is", "a", "nice", "day" , "nice", "."]
Counts_1 = {}
Counts_2 = {}
unigram_from_tokens_sen(tokens, Counts_1)
ngrams_from_tokens_sen(tokens, 2, Counts_2)
print(Counts_1)
print(Counts_2)

sumVals1 = sum_values(Counts_1, 1)
sumVals2 = sum_values(Counts_2, 2)
print(sumVals1)
print(sumVals2)

unigram_keser_nay = unigram_keser_nay_data(Counts_2)
print(unigram_keser_nay)

{'Hello': 1, 'world': 1, ',': 1, 'today': 1, 'is': 1, 'a': 1, 'nice': 2, 'day': 1, '.': 1}
{('w',): {'Hello': 1}, ('Hello',): {'world': 1}, ('world',): {',': 1}, (',',): {'today': 1}, ('today',): {'is': 1}, ('is',): {'a': 1}, ('a',): {'nice': 1}, ('nice',): {'day': 1, '.': 1}, ('day',): {'nice': 1}}
{'SUM': 10}
{('w',): 1, ('Hello',): 1, ('world',): 1, (',',): 1, ('today',): 1, ('is',): 1, ('a',): 1, ('nice',): 2, ('day',): 1}
{'total_bigrams': 10, 'Hello': 1, 'world': 1, ',': 1, 'today': 1, 'is': 1, 'a': 1, 'nice': 2, 'day': 1, '.': 1}
